# Glottolog Scraper

This file takes information from the downloaded glottolog files to obtain a glottocode and scrapes the corresponding Wikipedia link from that languages page on Glottolog.

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as py

status_data = pd.read_csv('../csv_files/glottolog_status_data.csv')

In [11]:
def get_links(): 
    link_base = 'https://glottolog.org/resource/languoid/id/'
    ids = pd.read_csv('../docs/master_merge.csv')['glottocode'].values

    links = []
    for i in tqdm(ids):
        links.append(link_base + i)
    return links

links = get_links()

100%|██████████| 7590/7590 [00:00<00:00, 4271403.11it/s]


In [14]:
# FILEPATH: /Users/gracewarren/LanguageStatusVisualizationProject/ipynb_files/glottolog_scraper.ipynb
# new_data = pd.DataFrame(columns = ['glottocode', 'aes_status'])

for link in tqdm(links):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')

    aes_status = soup.find('dt', string='AES status:')
    if aes_status is not None:
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'aes_status'] = aes_status.find_next_sibling('dd').string
    else:
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'aes_status'] = 'N/A'
        
    wiki_link = soup.find('a', title='Wikipedia')
    # print(wiki_link['href'])
    if wiki_link is not None:
        
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'Wikipedia_Url'] = wiki_link['href']
    else:
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'Wikipedia_Url'] = ''

status_data.to_csv('../csv_files/glottolog_status_data_with_links.csv', index=False)

100%|██████████| 7590/7590 [1:58:38<00:00,  1.07it/s]   
